In [ ]:
using System.IO;
using System.Diagnostics;
using Microsoft.DotNet.Interactive;

const string ffmpeg = @"ffmpeg.exe";
const string outputDir = @"I:\Temp\视频预览编码工具";

In [ ]:
var dir = new DirectoryInfo(Task.Run(async() => await Kernel.GetInputAsync("视频所在目录")).Result.Replace("\"", ""));
FileInfo[] files = dir.GetFiles("*.mp4", SearchOption.AllDirectories)
    .Union(dir.GetFiles("*.mkv", SearchOption.AllDirectories))
    .Union(dir.GetFiles("*.avi", SearchOption.AllDirectories))
    .Union(dir.GetFiles("*.flv", SearchOption.AllDirectories))
    .Union(dir.GetFiles("*.wmv", SearchOption.AllDirectories))
    .Union(dir.GetFiles("*.mov", SearchOption.AllDirectories))
    .Union(dir.GetFiles("*.rmvb", SearchOption.AllDirectories))
    .Union(dir.GetFiles("*.ts", SearchOption.AllDirectories)).ToArray();
var total = files.Length;
$"找到了 {files.Length} 个视频文件，共 {files.Sum(x=>x.Length)/(1024*1024*1024)}GB".Display();

In [ ]:
var count = 1;
foreach (var file in files)
{
    Console.WriteLine($"正在处理第 {count} / {total} 个");
    string outputFile = Path.Combine(outputDir, Path.GetFileNameWithoutExtension(file.FullName) + ".mp4");
    string arguments = $"-i \"{file}\" -vf \"scale=480:320\" -r 10 -c:v hevc_nvenc -b:v 200k -ac 6 \"{outputFile}\"";

    ProcessStartInfo startInfo = new ProcessStartInfo(ffmpeg, arguments)
    {
        UseShellExecute = true,
        CreateNoWindow = false,
        WindowStyle = ProcessWindowStyle.Minimized // 设置窗口为最小化
    };

    using (Process process = Process.Start(startInfo))
    {
        process.WaitForExit();
    }
    count++;
}